In [21]:
import numpy as np
import pandas as pd
import cvxpy as cp
import nashpy as nash
import pulp
from itertools import combinations

## Set-Up:

- Dataframe Loading
- Base Tradeoff Matrix
- Team - Vector Conversion Functions
- Example Teams

In [22]:
#Global Variables, don't change these, unless you are me:

payoff_df = pd.read_csv("MarvelRivals_Payoff_Matrix.csv")
hero_list = hero_list = list(payoff_df.iloc[:, 0])

payoff_df = pd.read_csv("MarvelRivals_Payoff_Matrix.csv",  index_col=0)
payoff_matrix = payoff_df.values

def team_to_vector(selected_heroes):
    vector = np.zeros(len(hero_list), dtype=int)
    for i, hero in enumerate(hero_list):
        if hero in selected_heroes:
            vector[i] = 1
    return vector
    
def vector_to_team(binary_vector):
    selected_heroes = [hero_list[i] for i in range(len(hero_list)) if binary_vector[i] == 1]
    return selected_heroes

Avengers = ["Black Widow", "Iron Man", "Thor", "Hulk", "Hawkeye", "Captain America"]
FantasticFour = ["Invisible Woman", "The Thing", "Human Torch", "Mister Fantastic", "Spider Man", "Wolverine"]
Guardians = ["Mantis", "Rocket Raccoon", "Adam Warlock", "Star Lord", "Groot", "Venom"]
Xmen = ["Magneto", "Psylocke", "Wolverine", "Magik", "Storm", "Namor"]
RandomTeam = ["Magneto", "Doctor Strange", "The Punisher", "Scarlet Witch", "Cloak & Dagger", "Loki"]

bestTeam = ['Captain America', 'Black Panther', 'Magik', 'Storm', 'Mantis', 'Rocket Raccoon']

In [23]:
MyTeam = ['Doctor Strange', 'Groot', 'The Punisher', 'Winter Soldier', "Cloak & Dagger", "Invisible Woman"]
OpponentTeam = ["Doctor Strange", "Peni Parker", "Magik", "Moon Knight", "Cloak & Dagger", "Invisible Woman"]

rowPlayer = team_to_vector(Guardians)
columnPlayer = team_to_vector(Guardians)

result = rowPlayer.T @ payoff_matrix @ columnPlayer
result

np.float64(0.0)

## Base Question Implementation: Optimal Conter Strategy given known opponent Strategy

It turns out, given an opponent team, it is relatively easy to figure out the optimal conter team using cvxpy.

In [24]:
# In order to use, first use team_to_vector() to convert the team into a binary vector
def base_optimal_conter_info(opponent_team):
    x = cp.Variable(37, boolean=True)
    y = opponent_team
    constraint_0 = [cp.sum(x) == 6]
    constraint_1 = [cp.sum(y) == 6]
    
    obj = cp.Maximize(x.T @ payoff_matrix @ y)
    prob = cp.Problem(obj,constraint_0+constraint_1)
    prob.solve()

    optimal_team = vector_to_team(x.value.round())
    expected_value = x.value.T @ payoff_matrix @ y

    print("Optimal Team:")
    for hero in optimal_team:
        print(f"-{hero}")
    print(f"Expected Utility Score of (x^T * A * y) is: {expected_value:.2f}")
    return x.value

def base_optimal_conter(opponent_team):
    x = cp.Variable(37, boolean=True)
    y = opponent_team
    constraint_0 = [cp.sum(x) == 6]
    constraint_1 = [cp.sum(y) == 6]
    
    obj = cp.Maximize(x.T @ payoff_matrix @ y)
    prob = cp.Problem(obj,constraint_0+constraint_1)
    prob.solve()

    return x.value

In [28]:
# Example Usage:
OpponentTeam = ["Doctor Strange", "Peni Parker", "Magik", "Moon Knight", "Cloak & Dagger", "Invisible Woman"]
y = team_to_vector(bestTeam)
print("Best Conter to OpponentTeam:", ', '.join(bestTeam), "is:")
OpponentTeamConter = base_optimal_conter_info(y)

Best Conter to OpponentTeam: Captain America, Black Panther, Magik, Storm, Mantis, Rocket Raccoon is:
Optimal Team:
-Captain America
-Iron Fist
-Magik
-Storm
-Mantis
-Rocket Raccoon
Expected Utility Score of (x^T * A * y) is: 0.35


## Secondary Question: Optimal Strategies for the payoff matrix

Since we have defined the row and column players and it clearly is a zero sum game, we can figure out he equilbrium state and its cooresponding optimal strategy of it, hence the "best team" highlightened in von neumann minimax theorem

In [ ]:
# 37 choose 6 is not very big so use Brutal Force Method:
def row_player_maximin_binary(payoff_matrix):
    n = payoff_matrix.shape[0]
    assert payoff_matrix.shape == (37, 37), "Expected a 37x37 payoff matrix."
    
    best_value = float('-inf')
    best_vector = None
    
    for row_subset in combinations(range(n), 6):
        c = np.sum(payoff_matrix[list(row_subset), :], axis=0)
        
        chosen_cols = np.partition(c, 6)[:6]
        payoff = np.sum(chosen_cols)
        
        if payoff > best_value:
            best_value = payoff
            vector = np.zeros(n, dtype=int)
            vector[list(row_subset)] = 1
            best_vector = vector

    print(f"Row-player guaranteed payoff is greater than: {best_value:.2f}")
    BestTeam = vector_to_team(vector)
    print("Von Neumann MinMax Best team is:", ', '.join(BestTeam))
    
    return best_vector

In [27]:
OptimalVector = row_player_maximin_binary(payoff_matrix)
print("Best Conter to von Neumann optimal team is:")
BestTeamConter = base_optimal_conter_info(OptimalVector)

Row-player guaranteed payoff is greater than: -0.12
Von Neumann MinMax Best team is: Captain America, Iron Fist, Magik, Storm, Mantis, Rocket Raccoon
Best Conter to von Neumann optimal team is:
Optimal Team:
-Captain America
-Peni Parker
-Magik
-Storm
-Mantis
-Rocket Raccoon
Expected Utility Score of (x^T * A * y) is: 0.12


## Secondary Question: Character Ban Stragegy Implementation

In the game, once you hit a certain rank (by being good at the game), the game introduce a new mechanism where a team can collectively ban two characters where the opponent team can no longer choose, this makes the character counter strategy more interesting.

## Secondary Question: Maximizing Team Ups

Problem: Maximize the number of possible team ups, with constraints:
- A team consists of 6 characters
- maximize win rate while as many heroes as possible is in a team up


team ups are powerful, but their strength is hard to quantify, therefore not very useful for solving the main problem: In this subproblem, we explore the ideal team composition where as many heroes as possible must be included in a team up while maintaining a standard team structure (2-3 strategists, 1-2 vanguards, 1-2 duelists) to maximize the team's win rate.

using the win rate of mathups

we were able to find 23 different teams with 5 team ups, which is the maximum number of team ups a team can get.

Using the matchup winrate matrix, we get this result: 
...


a team should consist of 2 of each class, but when this is enforced, we see that there is no team with 2 of each class with 5 team ups.


In [ ]:
team_ups = [
    ("Adam Warlock", "Star Lord"),
    ("Adam Warlock", "Mantis"),
    ("Thor", "Captain America"),
    ("Thor", "Storm"),
    ("Hela", "Loki"),
    ("Hela", "Thor"),
    ("Venom", "Spider Man"),
    ("Venom", "Peni Parker"),
    ("Hulk", "Doctor Strange"),
    ("Hulk", "Iron Man"),
    ("Rocket Raccoon", "The Punisher"),
    ("Rocket Raccoon", "Winter Soldier"),
    ("Invisible Woman", "The Thing"),
    ("Invisible Woman", "Mister Fantastic"),
    ("Magik", "Black Panther"),
    ("Magik", "Psylocke"),
    ("Human Torch", "Storm"),
    ("Iron Fist", "Luna Snow"),
    ("Spider Man", "Squirrel Girl"),
    ("Scarlet Witch", "Magneto"),
    ("Luna Snow", "Namor"),
    ("Luna Snow", "Jeff The Land Shark"),
    ("Groot", "Rocket Raccoon"),
    ("Groot", "Jeff The Land Shark"),
    ("Hulk", "Wolverine"),
    ("Invisible Woman", "Human Torch"),
    ("The Thing", "Wolverine"),
    ("Cloak & Dagger", "Moon Knight"),
    ("Hawkeye", "Black Widow")
]

win_rate_matrix = pd.read_csv("MarvelRivals_WinRate_matrix.csv", index_col=0)

heroes = [
    "Adam Warlock", "Black Panther", "Black Widow", "Captain America", "Cloak & Dagger",
    "Doctor Strange", "Groot", "Hawkeye", "Hela", "Hulk", "Human Torch",
    "Invisible Woman", "Iron Fist", "Iron Man", "Jeff The Land Shark", "Loki",
    "Luna Snow", "Magik", "Magneto", "Mantis", "Mister Fantastic",
    "Moon Knight", "Namor", "Peni Parker", "Psylocke", "Rocket Raccoon",
    "Scarlet Witch", "Spider Man", "Squirrel Girl", "Star Lord", "Storm",
    "The Punisher", "The Thing", "Thor", "Venom", "Winter Soldier", "Wolverine"
]

vanguards = ["Captain America", "Doctor Strange", "Groot", "Hulk", "Magneto", "Peni Parker", "The Thing", "Thor", "Venom"]
duelists = ["Black Panther", "Black Widow", "Hawkeye", "Hela", "Human Torch", "Iron Fist", "Iron Man", "Magik", "Mister Fantastic",
            "Moon Knight", "Namor", "Psylocke", "Scarlet Witch", "Spider Man", "Squirrel Girl", "Star Lord", "Storm", "The Punisher", 
            "Winter Soldier","Wolverine"]
strategists = ["Adam Warlock", "Cloak & Dagger", "Invisible Woman", "Jeff The Land Shark", "Loki", "Luna Snow", "Mantis", "Rocket Raccoon"]

if len(vanguards) + len(duelists) + len(strategists) != len(heroes):
    raise ValueError("The number of heroes is not correct")

hero_ids = {hero: idx for idx, hero in enumerate(heroes)}

vanguard_ids = [hero_ids[hero] for hero in vanguards]
duelist_ids = [hero_ids[hero] for hero in duelists]
strategist_ids = [hero_ids[hero] for hero in strategists]

team_up_tuples = [(hero_ids[a], hero_ids[b]) for a, b in sorted(team_ups) if hero_ids[a] < hero_ids[b]]

hero_avg_win_rates = win_rate_matrix.mean(axis=1).values

def printsolution(problem, x, y):
    selected_heroes = [heroes[i] for i in range(len(heroes)) if x.value[i] > 0.5]

    selected_team_ups = [(heroes[i], heroes[j]) for (i, j) in team_up_tuples if y[(i, j)].value > 0.5]

    # Calculate the average win rate using hero names
    selected_hero_avg_win_rates = [
    hero_avg_win_rates[i] for i in range(len(heroes)) if x.value[i] > 0.5
]

# Print the individual averages
    print("\nSelected Heroes' Average Win Rates:")
    for hero, rate in zip(selected_heroes, selected_hero_avg_win_rates):
        print(f"- {hero}: {rate:.2f}%")

    average_win_rate = problem.objective.value

    print("\nSelected Heroes:")
    for hero in selected_heroes:
        print("-", hero, ": Vanguard" if hero in vanguards else "Duelist" if hero in duelists else "Strategist")

    print("\nSelected Team-Ups:")
    for duo in selected_team_ups:
        print("-", duo)

    print(f"\nAverage Win Rate of Selected Team-Ups: {average_win_rate/600:.2%}")
    


In [ ]:
def solve_teamups(teamUpCount, classLimit = False):
    x = cp.Variable(len(heroes), boolean=True) 
    y = {t: cp.Variable(boolean=True) for t in team_up_tuples}

    objective = cp.Maximize(cp.sum(cp.multiply(x, hero_avg_win_rates)))
    
    constraints = [
        cp.sum(x) == 6,
        cp.sum([y[t] for t in team_up_tuples]) == teamUpCount
    ]
    if classLimit:
        constraints.extend([
        cp.sum(x[vanguard_ids]) >= 1,   
        cp.sum(x[vanguard_ids]) <= 2,
        
        cp.sum(x[duelist_ids]) >= 1,
        cp.sum(x[duelist_ids]) <= 2,
        
        cp.sum(x[strategist_ids]) <= 3   
        ])
    
    # boolean AND operation
    for (i, j) in team_up_tuples:
        constraints.append(y[(i, j)] <= x[i])
        constraints.append(y[(i, j)] <= x[j])
        constraints.append(y[(i, j)] >= x[i] + x[j] - 1)
    problem = cp.Problem(objective, constraints)
    problem.solve()
    print(problem.status)
    if problem.status == cp.OPTIMAL:
        printsolution(problem, x, y)

In [ ]:
solve_teamups(5, classLimit=False)

In [ ]:
solve_teamups(5, classLimit=True)

In [ ]:
solve_teamups(5, classLimit=False)

In [ ]:
solve_teamups(4, classLimit=True)